<a href="https://colab.research.google.com/github/Snrts/PRA3024_BigDataInPhysics_SanneAarts/blob/Week_4/SanneAartsML2023.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## The cartPole
### Action space
<ul>
<li>push left - 0</li>
<li>push right - 1</li>
</ul>

### Observation space
|num|Observation|Variable|Values|Episode terminates for:|
|-|-|-|-|-|
|0|Cart position|$x$|$\pm$ 4.8|$\lvert x \rvert > 2.4$|
|1|Cart Velocity|$\dot{x}$|$\pm \infty$||
|2|Pole Angle|$\theta$|$\pm$ 24 $^ \circ$|$\lvert \theta \rvert > 12 ^\circ $|
|3|Pole Angular Velocity|$\dot{\theta}$|$\pm\infty$||


In [4]:
import gym
import numpy as np
from matplotlib import pyplot as plt
import random

from collections import deque
from keras.models import Sequential, load_model
from keras.layers import Dense,  Flatten
from keras.optimizers import Adam, RMSprop
import os


#### Bellman Equation
At some time t, the expected return from an action a, taken from a starting state s, via some optimal policy, is given by the expected reward $R_{t+1}$
\begin{equation}
\begin{array}{cc}
Q_{new}(s, a)&=Q(s,a)+\alpha \left[ r(s,a)+ \gamma * max_{a'}(Q'(s',a'))-Q(s,a)\right]\\
\\
\text{                    }&=Q(s,a)\left(1-\alpha\right)+ \alpha \left[ r(s,a)+\gamma* max_{a'}(Q'(s',a'))\right] 
\end{array} \hspace{1em}
\begin{array}{rl}
Q_{new}:& \text{New Q value}\\
Q:& \text{Current Q value}\\
s:& \text{State}\\
a:& \text{Action}\\
\alpha:& \text{Learning Rate}\\
r:& \text{reward}\\
\gamma:& \text{Discount rate}\\
max(Q'(s',a')):& \text{Maximum expected future reward}\\
\end{array} 
\tag{1}
\end{equation}

The discount factor $\gamma$ ensures that the agent focusses on immediate rewards over future rewards, a lower value means more short term decisions. 

##### State value 
The expected total reward we can get being in a specific state
$$V^{\pi}(s)=\mathbb{E}\left[R_t | s_t = s\right]$$
##### Value function
The expected total reward we can get by performing some action starting from a state
$$Q^{\pi}(s,a)=\mathbb{E}\left[R_t | s_t = s, a_t=a\right]$$
#### Greedy Policy
A greedy policy is a policy where you always choose the most beneficial next step, 
in this case:
$$V(s_t)=max Q(s_t,a)$$
#### Loss function

\begin{equation}
L=\frac{1}{2}\left[r+\gamma max_{a'}(Q'(s',a'))-Q(s,a) \right]^2 
\tag{2}
\end{equation}

In [41]:
class DQNagent:
    def __init__(self, state_size, action_size):
        self.state_size = state_size
        self.action_size = action_size
        self.memory = deque(maxlen=100000)

        self.gamma = 0.95
        self.exploration_rate = 1.0
        self.exploration_rate_decay = 0.995
        self.exploration_rate_min = 0.01
        self.learning_rate = 0.001
        
        
        self.model = self.build_model()
        self.target = self.build_model()
        self.target.set_weights(self.model.get_weights())
        self.counter = 0
        self.c = 350
        self.loss = []
        self.trainingtime = []
        
    def build_model(self):
        # Building Network
        # Create a Sequential model using keras
        model = Sequential()
        #Input - 4 different states 
        # model.add(Flatten(input_shape=(1,self.state_size)))
        model.add(Flatten(input_shape = (self.state_size,)))
        model.add(Dense(24, activation='relu'))
        model.add(Dense(24, activation='relu'))
        # Output - one node for each possible action (2), the one with the 
        # highest output value will be be returned as action
        model.add(Dense(self.action_size, activation=None))
        model.compile(loss='mse', optimizer=Adam(lr=self.learning_rate))
        model.summary()
        return model
        
    def remember(self, state, action, reward, next_state, done): 
        self.memory.append((state, action, reward, next_state, done))
    
    def act(self, state):
        # Greedy Epsilon policy, a random number between 0 and 1 is selected, if the number is smaller
        # than the exploration rate,a random action is selected from the available actions
        if np.random.rand() <= self.exploration_rate:
            return random.randrange(self.action_size)
        # otherwise the action with the highest predicted Q-value is selected 
        # state is passed into the predict method to get the predicted Q value for
        # all possible actions (here, to go left or right)
        act_values = self.model.predict(state)
        # since we get two outputs, we want to select the action corresponding to the highest value
        return np.argmax(act_values[0])
    
    def decay_exploration(self):
        # As our model gets more experience, we want it to make more informed choices by using that experience, 
        # So while looping through different episodes we slowly decrease the exploration rate, since we don't want it to
        # go to 0 completely, we require that it remains above some previously defined minimum
        if self.exploration_rate > self.exploration_rate_min:
            self.exploration_rate *= self.exploration_rate_decay
    
    def train(self, batch_size):
        x,y = [], []
        if len(self.memory)>= batch_size:
            minibatch = random.sample(self.memory, batch_size)
            for index, (state, action, reward, next_state, done) in enumerate(minibatch):
                qvalue = self.model.predict(np.array([row[0] for row in minibatch]).reshape(-1,self.state_size ))[0]
                q_future = self.target.predict(np.array([row[3] for row in minibatch]).reshape(-1,self.state_size ))[0]
                # q_future = self.target.predict(next_state)[0]
                if done == True:
                    target = reward
                else:
                    target = reward + self.gamma * np.max(q_future)
                qcurr = qvalue[0]
                qcurr[action]=target
                x.append(state), y.append(qcurr)
            # x,y = np.array(x).reshape(1, batch_size, 1, self.state_size), np.array(y).reshape(1, batch_size, 1, self.action_size)
            # l = max(len(x),len(y))
            # x.resize(l), y.resize(l)
            loss = self.model.fit(x,y, batch_size=batch_size)
            self.loss.append(loss.history['loss'][0])
            if done:
                self.counter+=1

            if self.counter > self.c:
                self.target.set_weights(self.model.get_weights())
                self.counter = 0
    # def replay(self, batch_size, episode):
    #     # Once there are enough experiences in the memory to work with, 
    #     if len(self.memory) < batch_size:
    #         return
    #     # We take a random subset from the memory to learn from, it is important that it is random, sequential
    #     # runs are likely to be similar during later episodes 
    #     minibatch = random.sample(self.memory, batch_size)
    #     # For every collection of observables in he minibatch we calculate 
    #     for state, action, reward, next_state, terminal in minibatch:
    #         q_update = reward
            
    #         if not terminal:
    #             q_update = (reward + self.gamma * np.amax(self.model.predict(next_state)[0]))
            
    #         q_value = self.model.predict(state)
    #         q_value[0][action] = q_update
    #         history = self.model.fit(state, q_value, epochs=1, verbose=0)
    #         self.loss.append(history.history["loss"])
    #         self.trainingtime.append(history.history['loss'])
        

    
    def save(self):
        self.model.save_weights('dqn_weights.h5f', overwrite=True)
        # self.model.save_weights(name)
    
    def test(self,test_episodes = 100):
        env_test = gym.make('CartPole-v1')

        # Load the saved model weights
        model = Sequential([
            Dense(32, activation='relu', input_dim=env_test.observation_space.shape[0]),
            Dense(32, activation='relu'),
            Dense(env_test.action_space.n, activation=None)
        ])
        model.load_weights('dqn_weights.h5f')

        # Set the epsilon value to zero for testing
        explorationrate  = 0.0001

        # Run the test loop for 10 episodes
        test_episodes = 10
        test_rewards = []
        for i in range(test_episodes):
            state = env_test.reset()
            done = False
            ep_reward = 0
            while not done:
                # Use the learned policy to select actions
                q_values = model.predict(np.array([state]))[0]
                action = np.argmax(q_values)
                # Take the action and observe the new state and reward
                state, reward, done, _ = env_test.step(action)
                # Update the episode reward
                ep_reward += reward
            # Append the episode reward to the list of test rewards
            test_rewards.append(ep_reward)

        # Print the average reward obtained over all test episodes
        plt.plot(range(test_episodes), test_rewards)
        plt.axhline(sum(test_rewards)/len(test_rewards))
    



In [42]:
def cartpole():
    env = gym.make('CartPole-v1')
    state_size = env.observation_space.shape[0]
    action_size = env.action_space.n
    agent = DQNagent(state_size,action_size)
    episodes = 30
    batch_size=10
    episode_rewards = deque(maxlen=1000) 
    exp = agent.exploration_rate
    for episode in range(episodes):
        state = env.reset()
        state = np.reshape(state, [1,state_size])
        score = 0
        done = False
        while done != True:
            # steps += 1
            action = agent.act(state)
            next_state, reward, done, _ = env.step(action)
            
            reward = reward if not done else - reward
            next_state = np.reshape(next_state, [1,state_size])
            # self.agent.decay_exploration()
            agent.remember(state, action, reward, next_state, done)
            score += reward
            episode_rewards.append(score)
            state = next_state
            agent.decay_exploration()
            if len(agent.memory) > batch_size:
                agent.train(batch_size)
            
            if done:
                print("episode: {}/{}, score: {}, e: {:.2}".format(episode, episodes, score, agent.exploration_rate))
                
                break
    env.close()       
            
            

    # plt.plot(self.agent.exploration_rate)
    fig, axs = plt.subplots(2,2)

    axs[0,0].plot(episode_rewards)
    axs[0,1].plot(agent.exploration_rate)
    axs[1,0].plot(agent.loss)
    axs[1,1].plot(agent.trainingtime)
    # plt.ylabel("# actions before terminal")
    # plt.xlabel("run iteration")

def test(self):
    self.agent.test()
#     success_measure = np.mean(runsteplog[-15:])
#     print("Mean of last 15 runs: {0}".format(success_measure))
    
#     agent.save("weights")

In [43]:
cartpole()

Model: "sequential_18"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten_17 (Flatten)        (None, 4)                 0         
                                                                 
 dense_54 (Dense)            (None, 24)                120       
                                                                 
 dense_55 (Dense)            (None, 24)                600       
                                                                 
 dense_56 (Dense)            (None, 2)                 50        
                                                                 
Total params: 770
Trainable params: 770
Non-trainable params: 0
_________________________________________________________________
Model: "sequential_19"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten_18 (Flatten)        (No

TypeError: 'numpy.float32' object does not support item assignment

## Agent

$$loss=\left(r+\gamma max \hat{Q}(s,a')-Q(s,a\right)^2$$

In [ ]:
#create a vector of randomly generated parameter in the range [-1,1]
parameters = np.random.rand(4)*2-1

In [114]:
def run_episode(env, parameters):
    state = env.reset()
    totalreward = 0
    for _ in range(500):
        action = 0 if np.matmul(parameters, state) < 0 else 1
        state, reward, done, _ = env.step(action)
        reward = reward if not done else - reward
        totalreward += reward
        if done:
            break
    return totalreward

In [116]:
bestparams = None
bestreward = 0
env = gym.make('CartPole-v1')
for _ in range(10000):
    parameters = np.random.rand(4) * 2 - 1
    reward = run_episode(env,parameters)
    if reward > bestreward:
        bestreward = reward
        bestparams = parameters
        # considered solved if the agent lasts 200 timesteps
        if reward == 500:
            break
    print(bestparams)

[ 0.98154411 -0.09250534 -0.0068868   0.89230561]
[ 0.98154411 -0.09250534 -0.0068868   0.89230561]
[ 0.98154411 -0.09250534 -0.0068868   0.89230561]
[ 0.98154411 -0.09250534 -0.0068868   0.89230561]
[ 0.98154411 -0.09250534 -0.0068868   0.89230561]
[ 0.98154411 -0.09250534 -0.0068868   0.89230561]
[ 0.98154411 -0.09250534 -0.0068868   0.89230561]
[ 0.98154411 -0.09250534 -0.0068868   0.89230561]
[ 0.98154411 -0.09250534 -0.0068868   0.89230561]
[ 0.98154411 -0.09250534 -0.0068868   0.89230561]
[-0.21643228 -0.00933613  0.21403858  0.83584804]
[-0.21643228 -0.00933613  0.21403858  0.83584804]
[-0.21643228 -0.00933613  0.21403858  0.83584804]
[-0.21643228 -0.00933613  0.21403858  0.83584804]
[0.1355736  0.41995759 0.81964266 0.39888624]
[0.1355736  0.41995759 0.81964266 0.39888624]
[0.1355736  0.41995759 0.81964266 0.39888624]
[0.1355736  0.41995759 0.81964266 0.39888624]
[0.1355736  0.41995759 0.81964266 0.39888624]
[0.1355736  0.41995759 0.81964266 0.39888624]
[0.1355736  0.41995759 0